In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np


In [2]:
# Load VGG-16 with pretrained weights from ImageNet
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the convolutional base
for layer in base_model.layers:
    layer.trainable = False


In [3]:
# Add custom layers on top of VGG-16
x = Flatten()(base_model.output)
x = Dense(4096, activation='relu')(x)
x = Dense(4096, activation='relu')(x)
output_layer = Dense(1, activation='sigmoid')(x)  # For binary classification

# Create a new model
model = Model(inputs=base_model.input, outputs=output_layer)


In [4]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [5]:
# Load and preprocess image
img_path = 'aa.jpg'
img = load_img(img_path, target_size=(224, 224))
img_array = img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array = img_array / 255.0  # Normalize to [0,1]

# Predict using VGG-16
prediction = model.predict(img_array)
print("Prediction:", prediction)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 427ms/step
Prediction: [[0.32320476]]


In [6]:
# model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))